<a href="https://colab.research.google.com/github/Apoak/Deep-Learning-Projects/blob/main/Spam_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Your task in this lab is to set up and train a neural network on any dataset of your choosing.   Look at UCI ML Repo and Kaggle to find datasets, for example.  

Train a neural network on the dataset without any regularization or other special techniques to get a baseline train and test error.  Then see how much you can improve the network's test error through techniques learned in class like regularization, different optimizers, batch normalization, etc.

* Import data
* Format data
* Create test and training sets
* Train model

* Optomize later

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import sklearn
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
from google.colab import files
data = files.upload()

The last column of 'spambase.data' denotes whether the e-mail was considered spam (1) or not (0),

In [ ]:
from google.colab import files
names = files.upload()
filename = next(iter(names))

In [ ]:
stop_string = "1, 0.    | spam, non-spam classes"
names_list = []
with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            if stop_string in line:
                break
        line = file.readline()
        for line in file:
            title = line.strip().split(":")[0]
            names_list.append(title)
print(names_list)

In [ ]:
print(names)

In [ ]:
df = pd.read_csv('spambase.data', header=None, names = names_list)
print(df.shape)
df.head()



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# MAKE BATCH SIZE 57
X_values = df.iloc[:, :-1].values  # Features (all columns except last)
X_torch = torch.tensor(X_values).float()
y = df.iloc[:, -1].values   # Target (last column)
y_torch = torch.tensor(y).long()


X_train, X_test, y_train, y_test = train_test_split(X_torch, y_torch, test_size=0.2, random_state=42)

# # Split dataset: 80% training, 20% testing
# X_train, X_temp, y_train, y_temp = train_test_split(X_torch, y_torch, test_size=0.2, random_state=42)

# # Second split: Validation (10%) and Test (10%) from Temp
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
# X_train_torch = torch.tensor(X_train).float()
# X_test_torch = torch.tensor(X_test).float()
# y_train_torch = torch.tensor(y_train).long()
# y_test_torch = torch.tensor(y_test).long()
# print(X_train.shape)
# print(y.shape)

In [ ]:
improved_model = torch.nn.Sequential(
    torch.nn.Linear(56,100),
    torch.nn.SiLU(),
    torch.nn.Linear(100,100),
    torch.nn.SiLU(),
    torch.nn.Linear(100,100),
    torch.nn.SiLU(),
    torch.nn.Linear(100,100),
    torch.nn.SiLU(),
    torch.nn.Linear(100,2),
)

In [ ]:
basic_model = torch.nn.Sequential(
    torch.nn.Linear(56,100),
    torch.nn.SiLU(),
    torch.nn.Linear(100,100),
    torch.nn.SiLU(),
    torch.nn.Linear(100,100),
    torch.nn.SiLU(),
    torch.nn.Linear(100,100),
    torch.nn.SiLU(),
    torch.nn.Linear(100,2),
)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
lr=1e-2

In [ ]:
# Dataloader for training data
dataset = TensorDataset(X_train, y_train)
batch_size = 112
trainloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Dataloader for validation data
dataset = TensorDataset(X_val, y_val)
batch_size = 112
valloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Dataloader for test data
test_dataset = TensorDataset(X_test, y_test)
batch_size = 112
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float("inf")
        self.counter = 0
        self.stopped_epoch = None

    def check_stop(self, val_loss, epoch):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0  # Reset counter if loss improves
        else:
            self.counter += 1  # Increment counter if loss does not improve

        if self.counter >= self.patience:
            self.stopped_epoch = epoch
            return True  # Signal to stop training
        return False


In [ ]:
def basic_train_model(model, X_train, y_train, loss_func, opt, lr, epochs):
  for epoch in range(epochs):
      opt.zero_grad() # zero out the gradients
      z = model(X_train) # compute z values
      loss = loss_fn(z,y_train) # compute loss
      loss.backward() # compute gradients
      opt.step() # apply gradients
      print(f'epoch {epoch}: loss is {loss.item()}')

In [ ]:
early_stopping = EarlyStopping(patience=100)

In [ ]:
# FOR TRAINING MODELS
def train_model(model, dataloader, loss_func, opt, lr, epochs):
  train_mses = []
  test_mses = []
  best_test_mse = np.inf

  for epoch in range(epochs):
      model.train()
      for batch_idx, (batch_X, batch_y) in enumerate(dataloader):
        opt.zero_grad() # zero out the gradients
        z = model(batch_X) # compute z values
        loss = loss_fn(z,batch_y) # compute loss
        loss.backward() # compute gradients
        opt.step() # apply gradients


        train_mses.append(loss.item())

        model.eval()
        pred = model(X_test)
        test_mse = loss_fn(pred,y_test).item()

        if test_mse < best_test_mse:
          best_test_mse = test_mse
          torch.save(model.state_dict(), 'best.ckpt')

        test_mses.append(test_mse)
      print(f'epoch {epoch}: loss is {loss.item()}')

  model.load_state_dict(torch.load('best.ckpt', weights_only=True))

  model.eval()
  pred = model(X_train)
  best_train_mse = loss_fn(pred,y_train).item()

  pred = model(X_test)
  best_test_mse = loss_fn(pred,y_test).item()

  return train_mses, test_mses, best_train_mse, best_test_mse

In [ ]:
# FOR TESTING MODELS with test data
def model_accuracy(model, dataloader):

  model.eval()
  correct = 0
  total = 0
  count = 0

  with torch.no_grad():
      for batch_X, batch_y in dataloader:
          outputs = model(batch_X)
          _, predicted = torch.max(outputs, 1)
          total += batch_y.size(0)
          correct += (predicted == batch_y).sum().item()
          temp = correct/total
          print(f'epoch {count}: Accuracy: {temp}')
          count += 1

  accuracy = correct / total

  return accuracy

In [ ]:
# Basic model using SGD
opt = torch.optim.SGD(basic_model.parameters(), lr=lr)
basic_train_model(basic_model, X_train, y_train, loss_fn, opt, lr, epochs=200)

In [ ]:
# Accuracy calculation for basic network
y_pred = basic_model(X_test)
num_pred = y_pred.size(dim=0)
max_val, max_idx = torch.max(y_pred, dim = 1)

# Test Error
total = 0
for i in range(num_pred):
  if max_idx[i] == y_test[i]:
    total += 1

accuracy = total/num_pred
print(f'Accuracy of Torch NN on test data: {accuracy:.4f}')

In [ ]:
# Training error calculation for basic network
y_pred = basic_model(X_train)
num_pred = y_pred.size(dim=0)
max_val, max_idx = torch.max(y_pred, dim = 1)

# Training Error
total = 0
for i in range(num_pred):
  if max_idx[i] == y_train[i]:
    total += 1
accuracy = total/num_pred
error = 1- accuracy
print(f'Training error of Torch NN on training data: {error:.4f}')

In [ ]:
# Use Adam
opt = torch.optim.Adam(improved_model.parameters(), lr=lr, betas =(.9, .999), weight_decay= 0.001)
train_mses, test_mses, best_train_mse, best_test_mse = train_model(improved_model, trainloader, loss_fn, opt, lr, epochs = 42)

In [ ]:
# Test accuracy
nn_accuracy = model_accuracy(improved_model, testloader)
print(f'Accuracy of Torch NN: {nn_accuracy:.4f}')

In [ ]:
# Training accuracy
nn_accuracy_train = model_accuracy(improved_model, trainloader)
train_error = 1- nn_accuracy_train
print(f'Training error of Torch NN: {train_error:.4f}')

In [ ]:
def make_plot(train_mses,test_mses):
  plt.plot(train_mses)
  plt.plot(test_mses)
  plt.xlabel('Epoch')
  plt.ylabel('MSE')
  plt.legend(['Train','Test'])

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:

make_plot(train_mses,test_mses)
print(f'# parameters: {count_parameters(improved_model)}')
print(f'best train loss:{best_train_mse}')
print(f'best test loss:{best_test_mse}')